In [9]:
import random as rd
import unittest
import numpy as np
import itertools
import textdistance

# Topic of the project

### For this second project, I chose to play with strings and I propose to attempt to solve the following problems.

Note : During the project, we will apply continuous integration principles using CircleCI 

# 1- We consider words composed of lowercase alphabetic characters, separated by  a new line and we are interested in finding the 5 largest anagram groups

### Example :

#### Input
- undisplayed
- trace
- tea
- singleton
- eta
- eat
- displayed
- crate
- cater
- carte
- caret
- beta
- beat
- bate
- ate
- abet

#### Output

- Group of size 5: caret carte cater crate trace .
- Group of size 4: abet bate beat beta .
- Group of size 4: ate eat eta tea .
- Group of size 1: displayed .
- Group of size 1: singleton .

Source of this problem : http://poj.org/problem?id=2408

<hr/>

#### In order to solve this problem we will apply the oriented object principle learned in course.

I propose to consider the following classes.

The Class <strong>Word</strong> will represent a word . We can imagine the following methods.

- A method shuffle able to shuffle the letters in a word (useful for unitest)
- A method is_anagram able to tell us if a word is an anagram of an other one

The Class <strong>Dictionary</strong> will represent a dictionary constructed with a set of words inside which, there are anagrams. We can imagine the following methods:

- A method remove able to remove a word in a dictionary
- A method is_in_the_dictionary able to tell if a word is inside the dictionary we consider or not
- A method groups_of_anagrams able to find all the groups of anagrams included in the dictionary we consider.

The Class <strong>Problem</strong> will be used to resolve a problem of this type with a .txt file. We can imagine the following methods:
- A method parse able to transform a .txt file into a dictionary of words if it respects the format given by the statement
- A method resolve which use the .txt file in order to solve the problem (so it returns the groups of anagrams of the dictionary defined by the .txt file)

The Class <strong>TestMethods1</strong> will be used to check the good working of our methods from the different classes

In [42]:
class Word:
    def __init__(self,word):
        self.string = word
        
    ''' To check if two words are anagram of each other we will procede like this :

    First, we check if they have the same length. If it's the case...
    
    We take the first letter of the first word and we check if it's present in the second one.
    
    If it's not, we can conclude that the two words aren't anagrams 
    
    If it's the case, we delete the first occurence of the letter considered in the second word and we continue with the

    next letter of the first word. 
    
    Finally, we continue like that until the end of the first word considered.
    
    Note : We will not take in account the uppercase for this exercise and so we will consider that
    two words made of the exact same letters (even if some of them are maybe uppercase) are anagrams
    '''
        
    def is_anagram(self, word):
        first = self.string.lower()
        second = word.string.lower()
        length_1 = len(first)
        length_2 = len(second)
        result = False

        if length_1 == length_2:
            for i in range(0, length_1):
                if first[i] in second:
                    result = True
                    # We remove the first apparition of s1[i] in s2
                    second = second.replace(first[i], '', 1)
                else:
                    result = False
                    break
        return result

    def shuffle(self):
        # random.shuffle works on list so we decompose our word in a list of
        # letters
        s_list = list(self.string)
        rd.shuffle(s_list)
        string = "".join(s_list)
        word = Word(string)  # and then we recompose the word
        return word

    def __str__(self):
        return self.string

    def __repr__(self):
        return self.string

In [53]:
class Dictionary:
    def __init__(self,list_of_words):
        self.low = list_of_words
        
    def remove(self,word):
        list_of_words=self.low
        d_list=[w.string for w in list_of_words]
        if word.string in d_list:
            index=d_list.index(word.string)
            del list_of_words[index]
            self.low=list_of_words
    
    def is_in_the_dictionary(self,word):
        list_of_words=self.low
        d_list=[w.string for w in list_of_words]
        if word.string in d_list:
            return True
        else:
            return False
        
    ''' To find the group of anagrams we will procede like this :
     
     First, we create a copy of our dictionary.
     Then, for every word w of the initial dictionary,
         we are looking for all the words of the copied dictonary which are anagrams of the word w
         then,we put them into a list and we print them as wanted.
         
     In order to not have doublons at the end, we consider at each loop a list which we will contain,
     all the words which are anagrams of the current word w. At the end of the research of anagrams for 
     this word, we erase their existence in the copied dictionary.
     
     And, in order to not considered an anagram of a word already treated, we execute the loop only if 
     the word for which we are looking for anagrams is in the copied dictionary.
     
     Furthemore, at the end, we have to return only the 5 largest groups of anagrams.
         
    '''
        
    def groups_of_anagrams(self):
        groups=[]
        copy_low=self.low.copy()
        copy=Dictionary(copy_low)
        for word in self.low:
            if copy.is_in_the_dictionary(word):
                group=[]
                to_remove=[]
                for other_word in copy.low:
                    if other_word.is_anagram(word):
                        group.append(other_word)
                        to_remove.append(other_word)
                groups.append(group)
            for word_to_remove in to_remove:
                copy.remove(word_to_remove)
                
        groups_length=np.array([len(group) for group in groups])
        indexes=groups_length.argsort()[::-1][:5] #5 biggest elements of groups_length
        str=''
        for index in indexes:
             str=str+f"Group of size {groups_length[index]} : {groups[index]} \n"
        return str
    
    def __repr__(self):
        return str(self.low)

We can try with the example given in the question

In [60]:
l=["trace","tea","singleton","eta","eat","displayed","crate","cater","carte","caret","beta","beat","bate","ate","abet"]
dictionary=Dictionary([Word(i) for i in l])
 
print(dictionary.groups_of_anagrams())

Group of size 5 : [trace, crate, cater, carte, caret] 
Group of size 4 : [beta, beat, bate, abet] 
Group of size 4 : [tea, eta, eat, ate] 
Group of size 1 : [displayed] 
Group of size 1 : [singleton] 



Now, we have to consider that the input is not directly an object from the class Dictionary but rather a .txt file.

Each word will be separated by a new line as suggested in the question.

So, we need parser !

In [80]:
class Problem:
    def __init__(self,filename):
        self.filename=filename
    
    def parse(self,path = 'Oracles_1'):
         try:
            file = open(f"{path}/{self.filename}.txt", "r")
            number_of_words=len(file.readlines())
            file.seek(0)
            list_of_strings=[]
            for i in range(0,number_of_words):
                   list_of_strings.append(file.readline().replace('\n',''))
            file.close()
            return Dictionary([Word(i) for i in list_of_strings])
         except: 
            print("Please check the file (even it's name) , there is something wrong")
    
    def resolve_str(self,path='Oracles_1'):
        dictionary=self.parse(path)
        if dictionary!=None:
            return dictionary.groups_of_anagrams() #for the unittest

    def resolve(self,path='Oracles_1'):
        if self.resolve_str(path)!=None:
            print(self.resolve_str(path)) 
        

We can test it with the same dictionary as before

In [124]:
problem=Problem('Oracle1')
problem.resolve()

Group of size 5 : [trace, crate, cater, carte, caret] 
Group of size 4 : [beta, beat, bate, abet] 
Group of size 4 : [tea, eta, eat, ate] 
Group of size 1 : [displayed] 
Group of size 1 : [singleton] 



Now, we can apply it to a larger dictionary of words.

For instance, I propose to test it with some .txt files which are provided on the repo https://github.com/first20hours/google-10000-english

(used as a submodule)

In [152]:
problem_1=Problem('google-10000-english')
problem_1.resolve('google-10000-english')

Group of size 4 : [care, race, acer, acre] 
Group of size 4 : [edit, diet, tied, tide] 
Group of size 4 : [isp, psi, sip, ips] 
Group of size 4 : [post, stop, spot, tops] 
Group of size 4 : [spa, asp, sap, pas] 



In [155]:
problem_2=Problem('google-10000-english-usa-no-swears-long')
problem_2.resolve('google-10000-english')

Group of size 2 : [procedure, reproduce] 
Group of size 2 : [introduces, reductions] 
Group of size 2 : [permission, impression] 
Group of size 2 : [reduction, introduce] 
Group of size 2 : [statement, testament] 



In [156]:
problem_3=Problem('google-10000-english-usa-no-swears-medium')
problem_3.resolve('google-10000-english')

Group of size 3 : [meals, salem, males] 
Group of size 3 : [garden, danger, grande] 
Group of size 3 : [center, recent, centre] 
Group of size 3 : [least, tales, steal] 
Group of size 3 : [fears, fares, safer] 



In [157]:
problem_4=Problem('google-10000-english-usa')
problem_4.resolve('google-10000-english')

Group of size 4 : [post, stop, spot, tops] 
Group of size 4 : [care, race, acer, acre] 
Group of size 4 : [spa, asp, sap, pas] 
Group of size 4 : [edit, diet, tied, tide] 
Group of size 4 : [step, pets, sept, pest] 



It seems to work !

In [160]:
class TestMethods1(unittest.TestCase):

    def test_is_anagram(self):
        word_1 = Word('ChanCe')
        word_2 = Word('chnaec')
        word_3 = Word('kangaroo')
        word_4 = Word('KgroOanao')
        word_5 = Word('pneumonoultramicroscopicsilicovolcanoconiosis')
        word_6 = word_5.shuffle()
        word_7 = Word('Entertainment')
        word_8 = Word('Somethingelse')
        self.assertEqual(word_1.is_anagram(word_2), True)
        self.assertEqual(word_3.is_anagram(word_4), False)
        self.assertEqual(word_5.is_anagram(word_6), True)
        self.assertEqual(word_7.is_anagram(word_8), False)

    def test_groups_of_ancagrams(self):
        l=["trace","tea","singleton","eta","eat","displayed","crate","cater","carte","caret","beta","beat","bate","ate","abet"]
        dictionary=Dictionary([Word(i) for i in l])
        s='Group of size 5 : [trace, crate, cater, carte, caret] \nGroup of size 4 : [beta, beat, bate, abet] \nGroup of size 4 : [tea, eta, eat, ate] \nGroup of size 1 : [displayed] \nGroup of size 1 : [singleton] \n'
        self.assertEqual(dictionary.groups_of_anagrams()==s, True)
        
    def test_resolve(self):
        example=Problem('Oracle1')
        self.assertEqual(example.resolve_str()=='Group of size 5 : [trace, crate, cater, carte, caret] \nGroup of size 4 : [beta, beat, bate, abet] \nGroup of size 4 : [tea, eta, eat, ate] \nGroup of size 1 : [displayed] \nGroup of size 1 : [singleton] \n', True)
  
unittest.main(argv=[''], verbosity=1, exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 0.004s

OK


<hr>

## 2 - We want a program that will read in a dictionary and a list of phrases and determine which words from the dictionary, if any, form anagrams of the given phrases.

The program must find all sets of words in the dictionary which can be formed from the letters in each phrase. We will not include the set consisting of the original words. 

If no anagram is present, do not write anything, not even a blank line.


Input will consist of two parts. The first part is the dictionary, the second part is the set of phrases
for which you need to find anagrams. Each part of the file will be terminated by a line consisting of a
single ‘#’

Output will consist of a series of lines. Each line will consist of the original phrase, a space, an equal
sign (=), another space, and the list of words that together make up an anagram of the original phrase,
separated by exactly one space. These words must appear in alphabetic sequence. 

The sentences found must appear in alphabetic sequence too

### Example :

#### Input 

- ABC
- AND
- DEF
- DXZ
- K
- KX
- LJSRT
- LT
- PT
- PTYYWQ
- Y
- YWJSRQ
- ZD
- ZZXY

#
- ZZXY ABC DEF
- SXZYTWQP KLJ YRTD
- ZZXY YWJSRQ PTYYWQ ZZXY

#### Output

SXZYTWQP KLJ YRTD = DXZ K LJSRT PTYYWQ 

SXZYTWQP KLJ YRTD = DXZ K LT PT Y YWJSRQ

SXZYTWQP KLJ YRTD = KX LJSRT PTYYWQ ZD

SXZYTWQP KLJ YRTD = KX LT PT Y YWJSRQ ZD

Source of this problem : https://onlinejudge.org/index.php?option=onlinejudge&page=show_problem&problem=84

</hr>

<hr/>

#### In order to solve this problem we will used the previous work and, we will continue to use OOP principles

In this problem, we can extend the definition or anagrams for sentences. Two sentences made of words from the dictionary (input) are anagrams if they share the same letters.

So, we are looking for a way to generate anagrams of a given sentence knowing a dictionary.

However, we will consider only the phrases which are ordered in alphabetic sequence. (Ie, the words are ordered according to the alphabetic sequence)

This time, I propose to consider words as strings (thus, we will not create a specific class for the words)

I propose to consider the following classes.

The Class <strong>Sentence</strong> will represent a sentence constructed with a set of words. We can imagine the following methods.

- A method shuffle able to shuffle the words in a sentence (useful for unitest)
- A method tuple_to_sentence able to transform a tuple of words into a Sentence
- A method to_string able to transform the sentence in a string (separating the words with blank)
- A method to_sentence able to transform a string with blanks into a sentence made of words
- A method is_equivalent able to tell us if two sentences are composed of exactly the same words but not in the same order
- A method in_common able to tell us if a word have all its letters in common with a sentence
- A method is_anagram able to tell us if a sentence is an anagram of an other one


The Class <strong>Dictionary</strong> will represent a dictionary constructed with a set of words. We can imagine the following method:

- A method alphabetic which sort the words of the dictionary in alphabetic order.
- A method find_sentences which find all the sentences we can built with the words (ordered in alphabetic order) from the dictionary 
- A method all_anagrams able to give us all the anagrams of a sentence knowing the dictionary


The Class <strong>Problem</strong> will be used to resolve a problem of this type with a .txt file. We can imagine the following methods:
- A method parse able to transform a .txt file into a dictionary of words if it respects the format given previously
- A method resolve which use the .txt file in order to solve the problem (so it returns all the anagrams of a given sentence knowing the dictionary in the .txt file)

The Class <strong>TestMethods2</strong> will be used to check the good working of our methods from the different classes

In [1]:
class Sentence:
    def __init__(self,list_of_words):
        self.low = list_of_words
        
    def shuffle(self):
        low=self.low.copy()
        rd.shuffle(low)
        new_sentence=Sentence(low)
        return new_sentence
    
    def tuple_to_sentence(self,tup):
        low=[]
        for i in range(len(tup)):
            low.append(tup[i])
        self.low=low
    
    def to_string(self):
        if len(self.low)==0:
            print('your sentence is the empty sentence')
        else:
            string=''
            for word in self.low:
                string+=word+' ' #note that the final string will have a blank at the end (but it's not a probleme here)
            return string
    
    def to_sentence(self,string):
        self.low=string.split(sep=" ")
        
    def is_equivalent(self,sentence):
        low1=self.low.copy()
        low2=sentence.low.copy()
        low1.sort()
        low2.sort()
        return (low1==low2)
    
    def in_common(self,word):
        sentence=self.to_string().lower()
        word=word.lower() #we don't consider the uppercase
        bool=True
        for i in range(0,len(word)):
            bool=bool and (word[i] in sentence)
        return bool
    
    
    
    ''' To check if two sentences are anagram of each other we will procede like this :
     
    First, we tranform them into string but, we replace the spaces with nothing : The words are attached to each other.
    
    Example : Sentence("hello world") becomes "Helloworld"
    
    Then, we procede just as the first problem with the two strings generated before.

    We check if they have the same length. If it's the case...
    
    We take the first letter of the first string and we check if it's present in the second one.
    
    If it's not, we can conclude that the two sentences aren't anagrams 
    
    If it's the case, we delete the first occurence of the letter considered in the second string and we continue with the

    next letter of the first string. 
    
    Finally, we continue like that until the end of the first string considered.
    
    Note : We will not take in account the uppercase for this exercise and so we will consider that
    two sentences made of the exact same letters (even if some of them are maybe uppercases) are anagrams
    '''
    
    def is_anagram(self,Sentence2):
        first=self.to_string().replace(" ","")
        second=Sentence2.to_string().replace(" ","")
        length_1 = len(first)
        length_2 = len(second)
        result = False
        
        if length_1 == length_2:
            for i in range(0, length_1):
                if first[i] in second:
                    result = True
                    # We remove the first apparition of s1[i] in s2
                    second = second.replace(first[i], '', 1)
                else:
                    result = False
                    break
        return result
        
    def __repr__(self):
        return f"Sentence({str(self.low)})"

In [2]:
class Dictionary:
    def __init__(self,list_of_words):
        self.low = list_of_words
        
    def alphabetic(self):
        self.low.sort()
        
    ''' To find all the sentences we can built with the words (ordered in alphabetic order) from the dictionary,
     we start to order alphabeticly the dictionary we consider.
     
     Then, we create all the subsets of words ordered alphabeticly of our dictionary.
     
     For that, we will use list(itertools.combinations(dictionary, i)) which returns,
     all the subsets of words of size i of the dictionary, while preserving the order of the words existing within it. 
     
     This subsets are returned as tuples so we will have to use our function tuple_to_sentence in order to tranform them to sentences.
     
    '''
    
    def find_sentences(self):
        Sentences=[]
        self.alphabetic()
        dictionary=self.low
        for i in range(1,len(dictionary)):
            subsets=list(itertools.combinations(dictionary, i))
            for j in range(len(subsets)):
                sentence=Sentence([''])
                sentence.tuple_to_sentence(subsets[j])
                Sentences.append(sentence)
        return Sentences
    
    
    '''In order to find all the anagrams of a sentence, we will proceed like this :
    
    - First we create a dictionary made of all the words from our initial dictionary which share all their letters
    present in the sentence that we consider
    
    -Then, we create all the possible sentence from this new dictionary
    
    - At last, we create a list of anagrams which will contains all the sentences generated before which are anagrams of
    the sentence we entered and, which not share the same words with that sentence
    
    '''
    
    def all_anagrams(self,sentence):
        dictionary=self.low
        final_dictionary=[]
        anagrams=[]
        for word in dictionary:
            if sentence.in_common(word): #if the word share all its letters with the sentence
                final_dictionary.append(word)
                
        final_dictionary=Dictionary(final_dictionary)
        sentences=final_dictionary.find_sentences()
        
        for s in sentences:
            if s.is_anagram(sentence) and not(s.is_equivalent(sentence)): #we don't want to return the same words in the output
                anagrams.append(s.to_string())
        anagrams.sort()
        return anagrams
        
    def __repr__(self):
        return str(self.low)

In [7]:
class Problem:
    def __init__(self,filename):
        self.filename=filename
    
    def parse(self,path = 'Oracles_2'):
         try:
            file = open(f"{path}/{self.filename}.txt", "r")
            number_of_lines=len(file.readlines())
            file.seek(0)
            list_of_words=[]
            sentences=[]
            cpt=0
            current_word=file.readline().replace('\n','')
            while not ('#' in current_word): 
                list_of_words.append(current_word)
                current_word=file.readline().replace('\n','')
                cpt+=1
            
            for j in range(cpt,number_of_lines-1): #when we start to have the symbol "#"...
                sentence=Sentence(['']) #...we start considering sentences
                string=file.readline().replace('\n','')
                sentence.to_sentence(string)
                sentences.append(sentence)
            file.close()
            return Dictionary(list_of_words),sentences
         except: 
            print("Please check the file (even its name) , there is something wrong")
    
    def resolve_str(self,path='Oracles_2'):
        dictionary,sentences=self.parse(path)
        final_result=''
        if dictionary!=None:
            for i in range(len(sentences)):
                all_anagrams=dictionary.all_anagrams(sentences[i])
                for j in range(len(all_anagrams)):
                    final_result=final_result + f"{sentences[i].to_string()} =  {all_anagrams[j]}" + "\n" 
        return final_result
                
    def resolve(self,path='Oracles_2'):
        if self.resolve_str(path)!=None:
            print(self.resolve_str(path)) 

In [11]:
P=Problem('example')
P.resolve()

SXZYTWQP KLJ YRTD  =  DXZ K LJSRT PTYYWQ 
SXZYTWQP KLJ YRTD  =  DXZ K LT PT Y YWJSRQ 
SXZYTWQP KLJ YRTD  =  KX LJSRT PTYYWQ ZD 
SXZYTWQP KLJ YRTD  =  KX LT PT Y YWJSRQ ZD 



In [13]:
class TestMethods1(unittest.TestCase):

    def test_to_string(self):
        first_sentence=Sentence(['Hello','my','name','is','john'])
        self.assertEqual(first_sentence.to_string(),"Hello my name is john ")
        
    def test_to_sentence(self):
        string='Hello my name is john'
        sentence=Sentence([''])
        sentence.to_sentence(string)
        self.assertEqual(sentence.to_string(),string+' ')

    def test_in_common(self):
        first_sentence=Sentence(['Hello','my','name','is','john'])
        word1 = "hello"
        word2 = "abracadabra"
        self.assertEqual(first_sentence.in_common(word1), True)
        self.assertEqual(first_sentence.in_common(word2), False)
        
    def is_equivalent(self):
        sentence1=Sentence(['Hello','my','name','is','john'])
        sentence2=Sentence(['Hello','my','name','is','barbara'])
        self.assertEqual(sentence1.is_equivalent(sentence2), False)
        self.assertEqual(sentence1.is_equivalent(sentence1.shuffle()), False)
        
    def test_is_anagram(self):
        Sentence1=Sentence(["SXZYTWQP","KLJ","YRTD"])
        Sentence2=Sentence(["DXZ","K","LJSRT","PTYYWQ"])
        Sentence3=Sentence(['Hello','my','name','is','john'])
        Sentence4=Sentence1.shuffle()
        self.assertEqual(Sentence1.is_anagram(Sentence2),True)
        self.assertEqual(Sentence2.is_anagram(Sentence3),False)
        self.assertEqual(Sentence2.is_anagram(Sentence4),True)
        
    def test_all_anagrams(self):
        l=Dictionary(["ABC","AND","DEF","DXZ","K","KX","LJSRT","LT","PT","PTYYWQ","Y","YWJSRQ","ZD","ZZXY"])
        S=Sentence(["SXZYTWQP","KLJ","YRTD"])
        self.assertEqual(l.all_anagrams(S),['DXZ K LJSRT PTYYWQ ','DXZ K LT PT Y YWJSRQ ','KX LJSRT PTYYWQ ZD ','KX LT PT Y YWJSRQ ZD '])
        
    def test_resolve_str(self):
        P=Problem('example')
        self.assertEqual(P.resolve_str(),'SXZYTWQP KLJ YRTD  =  DXZ K LJSRT PTYYWQ \nSXZYTWQP KLJ YRTD  =  DXZ K LT PT Y YWJSRQ \nSXZYTWQP KLJ YRTD  =  KX LJSRT PTYYWQ ZD \nSXZYTWQP KLJ YRTD  =  KX LT PT Y YWJSRQ ZD \n')
        
unittest.main(argv=[''], verbosity=1, exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 0.034s

OK


<hr/>

# 3 - Finally, we will try to create a spell checker

We will study first, the correction of the mistakes in which all the letters of a word are present, but, not in the good order.

### Example :'Cra' instead of 'Car'

The program must find such mistakes in a sentence or a paragraph knowing a sets of words in a dictionary.

Moreover, I will extend the power of this spell checker in considering the case where some letters are forgotten in a word

### Exemple : 'Rom' instead of 'Room'

For instance, let's say that we want to also guess a word written by someone in which there are at most 2 letters missing (knowing a dictionary)


### Input : A .txt document with first, the words of the dictionary at each line until a line in which there will be the symbol "#" which will indicate that we start to consider sentences

- Movie
- Car
- Room
- Supermarket
- Computer
- #
- Today, I will go to the supemakret because I need a copmuter for my romo
- I am watching a moive from my cra

### Output : Proposition of corrections for the sentences

Sentence : 'Today, I will go to the supemakret because I need a copmuter for my romo'

I find errors, I think the correct answer is :

'Today, I will go to the supermarket because I need a computer for my room'

Sentence : 'I am watching a moive from my cra'

I find errors, I think the correct answer is :

'I am watching a movie from my car'

<hr/>

#### In order to solve this problem we will used the previous work and, we will continue to use OOP principles
#### Furthermore, we will use the Hamming distance as an indicator of a potential mistake in a sentence (It's a a string metric for measuring a distance between two sequences. More information here : https://en.wikipedia.org/wiki/Hamming_distance)

#### We will use this distance in order to guess a word which have at most 2 letters missing

#### So, if you remember what we have said before, we will looking for words in the dictionary, for which, the hamming distances between them and the incorrect words of our sentences are lower or equal than 2.

I propose to consider the following classes.


The Class <strong>Word</strong> will represent a word . We can imagine the following methods.

- A method shuffle able to shuffle the letters in a word (useful for unitest)
- A method is_anagram able to tell us if a word is an anagram of an other one
- A method hamming able to compute the hamming distance between two words
- A method correct able to propose correction for a word giving a dictionary

The Class <strong>Dictionary</strong> will represent a dictionary constructed with a set of words. We can imagine the following method:

- A method list_of_strings able to transform a dictionary made of Words in a list of strings
- A method incorrect_words which find all the words of a sentence which are not present in the dictionary

The Class <strong>Sentence</strong> will represent a sentence constructed with a set of words. We can imagine the following methods.

- A method correct_sentence which will propose all the corrections possible of a sentence giving a dictionary


The Class <strong>Problem</strong> will be used to resolve a problem of this type with a .txt file. We can imagine the following methods:
- A method parse able to transform a .txt file into a dictionary of words if it respects the format given previously
- A method resolve which use the .txt file in order to solve the problem (so it returns all the anagrams of a given sentence knowing the dictionary in the .txt file)

The Class <strong>TestMethods3</strong> will be used to check the good working of our methods from the different classes